In [ ]:
from pykml import parser
import folium
from shapely.geometry import LineString, Point, Polygon, shape
from pyproj import Transformer
import webbrowser
import geojson

INPUT & OUTPUT

In [ ]:
#Input
kml_file = 'Route.kml'
buffer_distance = 10 #buffer distance (meters)
tolerance = 0.00005 #tolerance

#Output
GeoJSON_file = 'Polygon.geojson'
HTML_file = 'Polygon.html'

KML

In [ ]:
with open(kml_file, mode='r', encoding='utf-8') as f:
    root = parser.parse(f).getroot()

coords = root.Document.Placemark.LineString.coordinates.text
coordinates = coords.strip().split() 

polyline_points = [] 
for coordinate in coordinates:
    lon, lat, __ = coordinate.split(',')
    latitude = float(lat)
    longitude = float(lon)
    polyline_points.append((latitude, longitude))

BUFFER & POLYGON

In [ ]:
in_proj = "EPSG:4326"
out_proj = "EPSG:3857"
transformer = Transformer.from_crs(in_proj, out_proj)
projected_points = [transformer.transform(lat, lon) for lat, lon in polyline_points]

line = LineString(projected_points)
buffered_line = line.buffer(buffer_distance)
buffer_polygon = buffered_line
buffer_polygon_exterior = buffer_polygon.exterior
buffer_polygon_coords = list(buffer_polygon_exterior.coords)
buffer_polygon_latlon = [transformer.transform(x, y, direction='INVERSE') for x, y in buffer_polygon_coords]

poly = Polygon(buffer_polygon_latlon)
simplified_poly = poly.simplify(tolerance, preserve_topology=True)
simplified_coords = list(simplified_poly.exterior.coords)

GeoJSON

In [ ]:
geojson_coords = [(lon, lat) for lat, lon in simplified_coords]
polygon = geojson.Feature(geometry=geojson.Polygon([geojson_coords]), properties={})

with open(GeoJSON_file, 'w') as f:
    geojson.dump(polygon, f)

parsing = {
    "type": polygon.geometry.type,
    "coordinates": polygon.geometry.coordinates
}
polygon_area = geojson.dumps(parsing)
print(polygon_area)

HTML

In [ ]:
m = folium.Map(location=polyline_points[0], zoom_start=12)
folium.Polygon(locations=buffer_polygon_latlon, color='red', fill=True, fill_color='red', fill_opacity=0.5).add_to(m)
folium.PolyLine(locations=polyline_points, color='red').add_to(m)
folium.GeoJson(polygon, name="geojson").add_to(m)
m.save(HTML_file)
m